# not working

In [1]:
from openai import OpenAI
from config.open_ai.dps_qc_report import get_qc_prompt

prompt = get_qc_prompt()

client = OpenAI()

assistant = client.beta.assistants.create(
  name="DPS QC Report Assistant",
  instructions=prompt,
  model="gpt-4o",
  tools=[{"type": "file_search"}],
)

In [2]:
# Create a vector store caled "DPS items"
vector_store = client.beta.vector_stores.create(name="DPS items")

from config.settings import DATA_DIR
import os

file_dir = DATA_DIR / 'qc_data' / 'qc_templates'
files = os.listdir(file_dir)
file_path = file_dir / files[-2]

# Ready the files for upload to OpenAI
file_paths = [file_path]
file_streams = [open(path, "rb") for path in file_paths]

# Use the upload and poll SDK helper to upload the files, add them to the vector store,
# and poll the status of the file batch for completion.
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)

# You can print the status and the file counts of the batch to see the result of this operation.
print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)


In [3]:
assistant = client.beta.assistants.update(
  assistant_id=assistant.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

In [4]:
# Upload the user provided file to OpenAI
message_file = client.files.create(
  file=open(file_path, "rb"), purpose="assistants"
)

# Create a thread and attach the file to the message
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": prompt,
      # Attach the new file to the message.
      "attachments": [
        { "file_id": message_file.id, "tools": [{"type": "file_search"}] }
      ],
    }
  ]
)

# The thread now has a vector store with that file in its tool resources.
print(thread.tool_resources.file_search)

ToolResourcesFileSearch(vector_store_ids=['vs_67a9da1ba1d8819191bc7f49d3779889'])


In [15]:
import json

# Fetch messages from the thread
messages = client.beta.threads.retrieve(thread.id)

from tqdm import tqdm
# Extract content from messages
for msg in tqdm(messages):
    print(msg)

5it [00:00, ?it/s]

('id', 'thread_driQfZoShy6piKeV37nHQyzZ')
('created_at', 1739184667)
('metadata', {})
('object', 'thread')
('tool_resources', ToolResources(code_interpreter=ToolResourcesCodeInterpreter(file_ids=[]), file_search=ToolResourcesFileSearch(vector_store_ids=['vs_67a9da1ba1d8819191bc7f49d3779889'])))


In [6]:
print(content[0].text.value)


You are an advanced Quality Control Report Extractor. Your task is to extract structured quality control details from 
the provided PDF and return a **list of valid JSON objects**, each strictly conforming to the schema below.

### **Instructions:**
1. **Skip the first few pages** containing summary reports.
2. **Each relevant page contains exactly one product's details.**
3. **Ignore pages that contain only images** or do not have structured text data.
4. **Extract only what is explicitly mentioned**; do not infer missing values.

### **Schema for Each Product:**
Each extracted product must contain the following fields exactly as they appear in the document:

- `product_name` (string): Name of the product.
- `RAG` (string): RAG value (e.g., "BLUE", "AMBER", "GREEN").
- `expected_qty` (string): Expected quantity.
- `received_qty` (string): Received quantity.
- `supplier_code` (string): Supplier code.
- `supplier` (string): Supplier name.
- `coo` (string): Country of Origin.
- `receive

In [ ]:
# Check the status of the assistant's task
assistant_status = client.beta.assistants.retrieve(assistant_id=assistant.id)
print(assistant_status.tool_resources)

ToolResources(code_interpreter=None, file_search=ToolResourcesFileSearch(vector_store_ids=['vs_67a9da148e0881919b9f9c23c272f610']))


In [11]:
msg

Message(id='msg_pcTOWFRSfJrTZbjWnePdcKT0', assistant_id=None, attachments=[Attachment(file_id='file-3LGXoaixV4Ci2Qqa7GvLFm', tools=[AttachmentToolAssistantToolsFileSearchTypeOnly(type='file_search')])], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='\nYou are an advanced Quality Control Report Extractor. Your task is to extract structured quality control details from \nthe provided PDF and return a **list of valid JSON objects**, each strictly conforming to the schema below.\n\n### **Instructions:**\n1. **Skip the first few pages** containing summary reports.\n2. **Each relevant page contains exactly one product\'s details.**\n3. **Ignore pages that contain only images** or do not have structured text data.\n4. **Extract only what is explicitly mentioned**; do not infer missing values.\n\n### **Schema for Each Product:**\nEach extracted product must contain the following fields exactly as they appear in the document:\n\n- `product_name` (string): Name of 